In [1]:
import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [2]:
t = read_mongo("wikidata", "triplet_train")

In [3]:
by_predicate = t.groupby(by=['Predicate','PredicateId'],as_index=False).agg({'Object':'count'})
by_predicate['Count'] = by_predicate['Object']
del by_predicate['Object']

In [4]:
p = read_mongo("wikidata", "property")
p.head()

,Qualifier,ReadTitleUk,SectionId,Title,Used,WikidataId
0,None,член політичної партії,person,member of political party,True,P102
1,None,локація штаб-квартири,organization,headquarters location,True,P159
2,None,локація формації,organization,location of formation,True,P740
3,None,головний виконавчий директор,organization,chief executive officer,True,P169
4,None,представляє організацію,organization,represents organisation,True,P1268


In [11]:
by_predicate = by_predicate.sort_values(by='Count', axis=0, ascending=False)
by_predicate

,Predicate,PredicateId,Count
18,знаходиться в адміністративно-територіальній о...,P131,278558
21,країна,P17,276004
26,містить адміністративно-територіальну одиницю,P150,90158
0,адміністративний центр,P36,23349
20,континент,P30,5853
12,дитина,P40,4960
5,брат,P7,4601
4,батько,P22,4148
35,партнер у шлюбі,P26,4091
24,локація штаб-квартири,P159,3559


In [6]:
pp = p.merge(by_predicate, how='left', left_on='WikidataId', right_on='PredicateId')
pp.head()

,Qualifier,ReadTitleUk,SectionId,Title,Used,WikidataId,Predicate,PredicateId,Count
0,None,член політичної партії,person,member of political party,True,P102,член політичної партії,P102,1526.0
1,None,локація штаб-квартири,organization,headquarters location,True,P159,локація штаб-квартири,P159,3559.0
2,None,локація формації,organization,location of formation,True,P740,локація формації,P740,473.0
3,None,головний виконавчий директор,organization,chief executive officer,True,P169,головний виконавчий директор,P169,8.0
4,None,представляє організацію,organization,represents organisation,True,P1268,представляє організацію,P1268,1.0


In [7]:
del pp['Qualifier']
del pp['ReadTitleUk']
del pp['SectionId']
del pp['Used']

pp['TitleEn'] = pp['Title']
del pp['Title']

pp['TitleUk'] = pp['Predicate']
del pp['Predicate']

pp['Id'] = pp['WikidataId']
del pp['WikidataId']
del pp['PredicateId']
pp.head()

,Count,TitleEn,TitleUk,Id
0,1526.0,member of political party,член політичної партії,P102
1,3559.0,headquarters location,локація штаб-квартири,P159
2,473.0,location of formation,локація формації,P740
3,8.0,chief executive officer,головний виконавчий директор,P169
4,1.0,represents organisation,представляє організацію,P1268


In [8]:
pp['Count'] = pp['Count'].fillna(0)
pp['Count'] = pp['Count'].astype(int)

In [9]:
pp = pp.sort_values(by='Count', axis=0, ascending=False)
pp.head()

,Count,TitleEn,TitleUk,Id
52,278558,located in the administrative territorial entity,знаходиться в адміністративно-територіальній о...,P131
51,276004,country,країна,P17
54,90158,contains administrative territorial entity,містить адміністративно-територіальну одиницю,P150
6,23349,capital,адміністративний центр,P36
50,5853,continent,континент,P30


In [10]:
pp.to_csv('D:\DRIVE\MS CS UCU\Machine Learning\Project\reports\by-predicate-count.csv', index=False, encoding='utf8', sep='\t')

OSError: [Errno 22] Invalid argument: 'D:\\DRIVE\\MS CS UCU\\Machine Learning\\Project\reports\x08y-predicate-count.csv'

In [ ]:
t['Len'] = t.apply(lambda row: len(row['Text']),axis=1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
t['Len'].hist(bins=80)

In [ ]:
len(t[t.Len > 1000])

In [ ]:
t[t.Len < 500]['Len'].hist(bins=80)